# Access SWOT HR data

![](https://img.shields.io/badge/CNES-tutorial-%20?color=blue&labelColor=grey)

_Authors: Mathilde Siméon, Damien Desroches (CNES)_
 
### Objectives
This tutorial explains  
* Where to find SWOT products
* How to search SWOT products on Hydroweb.Next and PO.DAAC
* How to download files on local disk

### Requirements
1. Install the `swot-training` python environment with `eodag` and `earthaccess` installed 
2. Configure `eodag` with hydroweb.next authentification
3. Run the notebook in `swot-training` environment


### Content
* [Products overview](#Products-overview)
* [Setup](#Setup)
* [Get PIXC, river products and PLD from Hydroweb Next](#Get-HR-products-and-PLD-from-Hydroweb.next)
* [Get SLC products from PO.DAAC](#Get-SLC-products-from-po.daac)


## Products overview


SWOT measurements over lake and rivers are available on [Hydroweb next](https://hydroweb.next.theia-land.fr/)  as well as PO.DAAC  and [NASA Earthdata Cloud](https://search.earthdata.nasa.gov/search?q=SWOT_*_2.0). 

Due to the platform's differences, some discrepancies in collections ID can appear. 
The table below gives a synthetic view of the products ID and  format for both platforms.


### L1 products

| Product   |      Hydroweb Next   ID   |  PO.DAAC ID  |Format |
|----------|-------------|------------|------------|
| SWOT L1 Single Look Complex Data product |   &cross; |  SWOT_L1B_HR_SLC_2.0   | netCDF-4 |

### L2 products
Level-2 products provide measurements over water observations only

#### Ungeolocated image
Ungeolocated image, with pixel classification

| Lake Products   |      Hydroweb Next ID      |  PO.DAAC ID |Format |
|----------|-------------|------|------|
| Point cloud of water mask pixels | SWOT_L2_HR_PIXC |  SWOT_L2_HR_PIXC_2.0   | netCDF-4 |
| Auxiliary information for pixel cloud product | SWOT_L2_HR_PIXCVEC | SWOT_L2_HR_PIXCVec_2.0 | netCDF-4 |

#### Lake Single Pass Vector

| Lake Products   |      Hydroweb Next ID      |  PO.DAAC ID |Format |
|----------|-------------|------|------|
|  | SWOT_L2_HR_LAKESP_OBS | SWOT_L2_HR_LakeSP_obs_2.0|  ESRI Shapefile |
|  | SWOT_L2_HR_LAKESP_PRIOR | SWOT_L2_HR_LakeSP_prior_2.0|   ESRI Shapefile |
|  | SWOT_L2_HR_LAKESP_UNASSIGNED |  SWOT_L2_HR_LakeSP_unassigned_2.0|  ESRI Shapefile |

#### River Single Pass Vector

| River Products   |      Hydroweb Next ID     |  PO.DAAC ID |Format |
|----------|-------------|------|------|
|L2 River Reaches Product | SWOT_L2_RIVERSP_NODE   | SWOT_L2_HR_RiverSP_node_2.0 |   ESRI Shapefile |
|L2 River Nodes Product | SWOT_L2_RIVERSP_REACH  | SWOT_L2_HR_RiverSP_reach_2.0|  ESRI Shapefile | 

#### Fixed grid

|Raster products    |      Hydroweb Next      |  PO.DAAC | Format |
|----------|-------------|------|------|
|  | L2_HR_RASTER_100M | SWOT_L2_HR_Raster_100m_2.0 | netCDF-4 |
|  |L2_HR_RASTER_250M |  SWOT_L2_HR_Raster_250m_2.0 | netCDF-4 | 

#### Ancillary files
| Ancillary Products   |      Hydroweb Next ID     |  PO.DAAC ID |Format |
|----------|-------------|------|------|
| SWOT Prior Lake Database          | SWOT_PRIOR_LAKE_DATABASE | &cross; |  ESRI Shapefile | 
| SWOT Prior River Database - SWORD | SWOT_PRIOR_RIVER_DATABASE | &cross; |   ESRI Shapefile | 


 
 

## Setup
This section imports the necesary librairies and defines some useful functions

In [1]:
import os

import earthaccess
import eodag
import folium
import shutil

In [2]:
def check_catalog(catalog):
    if catalog not in dag.available_providers():
        raise Warning(
            "It seems that hydroweb_next authentification is not found in ~/.config/eodag/eodag.yml"
        )
    else:
        print(f"{catalog} is available in eodag")

In [3]:
def display_search_zone(geometry):
    # Create a map zoomed over the search area
    fmap = folium.Map([geometry["latmin"], geometry["lonmax"]], zoom_start=6)
    # display a red rectangle
    folium.Rectangle(
    bounds=[
        [geometry["latmin"], geometry["lonmin"]],
        [geometry["latmax"], geometry["lonmax"]],
    ],
    color="red",
    ).add_to(fmap)   
    return fmap

Define a directory where to download the data

In [5]:
workdir = "./data"
if not os.path.exists(workdir):
    os.makedirs(workdir)

## Get HR products and PLD from Hydroweb.next
### Connect
`EODAG` python library is used to query the hydroweb next catalog. We will see how to connect to a specific catalog to explore it.

More information about the eodag library is documented on the following [documentation](https://eodag.readthedocs.io/en/stable/notebooks/api_user_guide/1_overview.html)


In [7]:
dag = eodag.EODataAccessGateway()

2024-09-06 20:34:16,884 - eodag.config - INFO - Loading user configuration from: /home/qt/simeonma/.config/eodag/eodag.yml
2024-09-06 20:34:16,900 - eodag.config - INFO - mundi: unknown provider found in user conf, trying to use provided configuration
2024-09-06 20:34:16,901 - eodag.config - WARNING - mundi skipped: could not be loaded from user configuration
2024-09-06 20:34:16,911 - eodag.core - INFO - usgs: provider needing auth for search has been pruned because no crendentials could be found
2024-09-06 20:34:16,912 - eodag.core - INFO - aws_eos: provider needing auth for search has been pruned because no crendentials could be found
2024-09-06 20:34:16,913 - eodag.core - INFO - meteoblue: provider needing auth for search has been pruned because no crendentials could be found
2024-09-06 20:34:16,915 - eodag.core - INFO - creodias_s3: provider needing auth for search has been pruned because no crendentials could be found
2024-09-06 20:34:16,916 - eodag.core - INFO - swot_ce: provider

Let's check that hydroweb_next authentification is provided and the catalog is available

In [8]:
check_catalog("hydroweb_next")

hydroweb_next is available in eodag


If eodag returns the following message for a provider, you need to check that eodag configuration is correct for this provider.

<div class="alert alert-warning">
     2024-01-31 14:46:25,916 - eodag.core - INFO - hydroweb_next: provider needing auth for search has been pruned because no crendentials could be found
</div>


### Define a search area
Let's define a search box around Pemproke in Canada

In [9]:
search_box = {"lonmin": -78, "latmin": 44.5, "lonmax": -76, "latmax": 46.5}

In [10]:
fmap = display_search_zone(search_box)
fmap

### Search and download SWOT PIXC product

Info: it is possible to get the data type 
with the following line 
`[ x['ID'] for x in dag.list_product_types(provider="hydroweb_next") ]`

In [11]:
catalog = "hydroweb_next"

We want to find SWOT products around the Ottawa river, so we will use the search geometry to find every product intersecting the search area within the time range

<div class="alert alert-info">
     <b>Note</b> If you know exactly which tile you want, you can search it with the spatial keywords
    
    'spatial:tile_side': 'Left'
    'spatial:tile_id': 74

</div>



In [12]:
search_criteria = {
    "provider": catalog,
    "productType": "SWOT_L2_HR_PIXC",
    "geom": search_box,
    "start": "2023-06-13T04:03:00",
    "end": "2023-06-13T04:30:00",
}
pixc_products, nb = dag.search(**search_criteria)
pixc_products

2024-09-06 20:35:07,735 - eodag.core - WARNING - unknown product type SWOT_L2_HR_PIXC
2024-09-06 20:35:07,736 - eodag.config - INFO - Fetching external product types from /work/softs/vre/projets/swotce/exploitation/envs/ceswot-stable/lib/python3.10/site-packages/eodag/resources/ext_product_types.json
2024-09-06 20:35:08,292 - eodag.search.qssearch - INFO - Fetching product types: https://hydroweb.next.theia-land.fr/api/v1/rs-catalog/stac/search/../collections
2024-09-06 20:35:10,809 - eodag.search.qssearch - INFO - Fetching product types: https://regards-swot.sis.cnes.fr/api/v1/rs-catalog/stac/collections/dynamic
2024-09-06 20:35:12,118 - eodag.core - INFO - Searching product type 'SWOT_L2_HR_PIXC' on provider: hydroweb_next
2024-09-06 20:35:12,122 - eodag.search.qssearch - INFO - Sending search request: https://hydroweb.next.theia-land.fr/api/v1/rs-catalog/stac/search
2024-09-06 20:35:12,286 - eodag.core - INFO - Found 10 result(s) on provider 'hydroweb_next'


SearchResult([EOProduct(id=URN:FEATURE:DATA:swot:c655478b-a080-343d-8a5a-c67f14358771:V1, provider=hydroweb_next),
              EOProduct(id=URN:FEATURE:DATA:swot:06cc27f1-e473-331f-8077-912abf295ec9:V1, provider=hydroweb_next),
              EOProduct(id=URN:FEATURE:DATA:swot:5fdb723d-52e3-3ec3-b4b3-9d5900e224c5:V1, provider=hydroweb_next),
              EOProduct(id=URN:FEATURE:DATA:swot:cac56f02-bf56-3719-90cc-bc0e453de857:V1, provider=hydroweb_next),
              EOProduct(id=URN:FEATURE:DATA:swot:443db824-1387-3f67-a751-f2039210bd71:V1, provider=hydroweb_next),
              EOProduct(id=URN:FEATURE:DATA:swot:ab35e070-fc24-3ce0-835f-78031eca633c:V1, provider=hydroweb_next),
              EOProduct(id=URN:FEATURE:DATA:swot:3464ba4a-4837-39e9-8317-f9d5ef771b27:V1, provider=hydroweb_next),
              EOProduct(id=URN:FEATURE:DATA:swot:1b1f19ef-0a9a-3b7c-9f22-2f343d92caec:V1, provider=hydroweb_next),
              EOProduct(id=URN:FEATURE:DATA:swot:9016f221-69e6-3614-8807-66afed4

#### Visualize

In [13]:
# Create a layer that maps the products found
folium.GeoJson(
    data=pixc_products,
    tooltip=folium.GeoJsonTooltip(
        fields=[
            "title",
            "spatial:cycle_id",
            "spatial:pass_id",
            "spatial:tile_side",
            "spatial:tile_id",
        ]
    ),
).add_to(fmap)
fmap

<div class="alert alert-info">
     <b>TIP !</b> It is possible to save the interactive map for presentation purposes or later use
</div>



In [22]:
fmap.save("Pembroke_pixc_results.html")

From the look on the interactive map, the tile 74 (Left) seems to cover an interesting part of the river.

We filter the search results such as only the products with a tile ID of 74 and the side is 'Left' is kept.

In [15]:
fpixc_products = []
for product in pixc_products:
    if (
        product.properties["spatial:tile_id"] == 74
        and product.properties["spatial:tile_side"] == "Left"
    ):
        fpixc_products.append(product)
fpixc_products

[EOProduct(id=URN:FEATURE:DATA:swot:06cc27f1-e473-331f-8077-912abf295ec9:V1, provider=hydroweb_next)]

#### Download specific assets
For some products like SWOT_L2_HR_PIXC, a search results contains other static files along with the product.
In this case, the product is accompanied with json and xml that are not relevant to the study. 

In [33]:
list(fpixc_products[0].assets)

['SWOT_L2_HR_RiverSP_Node_478_016_EU_20230402T101945_20230402T101950_PGC0_01.zip.md5',
 'SWOT_L2_HR_RiverSP_Node_478_016_EU_20230402T101945_20230402T101950_PGC0_01.zip.iso.xml.md5',
 'SWOT_L2_HR_RiverSP_Node_478_016_EU_20230402T101945_20230402T101950_PGC0_01.log.md5',
 'SWOT_L2_HR_RiverSP_Node_478_016_EU_20230402T101945_20230402T101950_PGC0_01.met.json',
 'SWOT_L2_HR_RiverSP_Node_478_016_EU_20230402T101945_20230402T101950_PGC0_01.rc.xml',
 'SWOT_L2_HR_RiverSP_Node_478_016_EU_20230402T101945_20230402T101950_PGC0_01.png.md5',
 'SWOT_L2_HR_RiverSP_Node_478_016_EU_20230402T101945_20230402T101950_PGC0_01.zip',
 'SWOT_L2_HR_RiverSP_Node_478_016_EU_20230402T101945_20230402T101950_PGC0_01.zip.iso.xml',
 'SWOT_L2_HR_RiverSP_Node_478_016_EU_20230402T101945_20230402T101950_PGC0_01.log',
 'SWOT_L2_HR_RiverSP_Node_478_016_EU_20230402T101945_20230402T101950_PGC0_01.met.json.md5',
 'SWOT_L2_HR_RiverSP_Node_478_016_EU_20230402T101945_20230402T101950_PGC0_01.rc.xml.md5',
 'SWOT_L2_HR_RiverSP_Node_478_0

Let's only download the netCDF product by setting a regular expression matching the asset we want to download

In [16]:
dag.download_all(
    fpixc_products, asset=r"SWOT_L2_HR_PIXC_.*.nc", outputs_prefix=workdir
)

2024-09-06 20:39:48,233 - eodag.core - INFO - Downloading 1 products


Downloaded products:   0%|          | 0/1 [00:00<?, ?product/s]

0.00B [00:00, ?B/s]

2024-09-06 20:39:48,258 - eodag.download.base - INFO - Download url: https://hydroweb.next.theia-land.fr/api/v1/rs-catalog/stac/collections/SWOT_L2_HR_PIXC/items/URN:FEATURE:DATA:swot:06cc27f1-e473-331f-8077-912abf295ec9:V1?token=eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJmSkotTTRGUU5QM0l5VzNzaXFZTDdwSHg4dVNpMENnbmZIY2ZTd1NxUlZVIn0.eyJleHAiOjE3MjU2NTUyMDgsImlhdCI6MTcyNTY1NDkwOCwianRpIjoiMTM3MjRmMTctNzVhYi00YTkzLWIwOTMtMTNhZDE3NWM3OTdiIiwiaXNzIjoiaHR0cHM6Ly9obmcuaHlzb3BlMi5jbmVzLmZyL2F1dGgvcmVhbG1zL2h5c29wZSIsImF1ZCI6ImhuZy1hcGkiLCJzdWIiOiJlMWI4Mjc3Yi1jYjI5LTQxMTctYjNkOS01NmM4OTM0YzE3MGYiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJobmctYXBpIiwic2Vzc2lvbl9zdGF0ZSI6ImY1YThmZmFkLTNmYzgtNDUwNS1hY2U2LTZiNWFiZTEwMWE0ZSIsImFjciI6IjEiLCJhbGxvd2VkLW9yaWdpbnMiOltdLCJzY29wZSI6Im1pY3JvcHJvZmlsZS1qd3QgcmVnYXJkc19zY29wZSBlbWFpbCIsInNpZCI6ImY1YThmZmFkLTNmYzgtNDUwNS1hY2U2LTZiNWFiZTEwMWE0ZSIsInVwbiI6Im1hdGhpbGRlLnNpbWVvbkBjbmVzLmZyIiwicm9sZSI6IlJFR0lTVEVSRURfVVNFUiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJhY2Nlc3NHcm

['/work/EXPERT_CENTER/mwec/workspace/HR/tuto_ostst/data2/URN_FEATURE_DATA_swot_06cc27f1-e473-331f-8077-912abf295ec9_V1-URN_FEATURE_DATA_swot_06cc27f1-e473-331f-8077-912abf295ec9_V1']

### Search and download Prior Lake Database
Note there is no need for time keywords because the PLD is a static database that does not evolve in time.
It is provided as basin-grouped files

In [18]:
search_criteria = {
    "provider": catalog,
    "productType": "SWOT_PRIOR_LAKE_DATABASE",
    "geom": search_box,
}

pld_results= dag.search_all(**search_criteria)

2024-09-06 20:40:41,763 - eodag.core - INFO - Searching product type 'SWOT_PRIOR_LAKE_DATABASE' on provider: hydroweb_next
2024-09-06 20:40:41,764 - eodag.core - INFO - Iterate search over multiple pages: page #1
2024-09-06 20:40:41,765 - eodag.search.qssearch - INFO - Sending search request: https://hydroweb.next.theia-land.fr/api/v1/rs-catalog/stac/search
2024-09-06 20:40:41,933 - eodag.core - INFO - Found 1 result(s) on provider 'hydroweb_next'


We can see that the Prior Lake database file corresponding to the Basin ID containing the search box is found

In [19]:
# Add  another layer that maps the products found
folium.GeoJson(
    data=pld_results,
).add_to(fmap)
fmap

In [20]:
dag.download_all(pld_results, outputs_prefix=workdir)

2024-09-06 20:42:03,003 - eodag.core - INFO - Downloading 1 products


Downloaded products:   0%|          | 0/1 [00:00<?, ?product/s]

0.00B [00:00, ?B/s]

2024-09-06 20:42:03,016 - eodag.download.base - INFO - Download url: https://hydroweb.next.theia-land.fr/api/v1/rs-catalog/stac/collections/SWOT_PRIOR_LAKE_DATABASE/items/URN:FEATURE:DATA:hysope2:7df60ccf-2b93-388d-be2e-7734fa7fea99:V1?token=eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJmSkotTTRGUU5QM0l5VzNzaXFZTDdwSHg4dVNpMENnbmZIY2ZTd1NxUlZVIn0.eyJleHAiOjE3MjU2NTU0ODgsImlhdCI6MTcyNTY1NTE4OCwianRpIjoiYjZlYjkzYzgtNWMwOC00YWQ0LTk4ZWYtNzUwOWI2NGJlMDBmIiwiaXNzIjoiaHR0cHM6Ly9obmcuaHlzb3BlMi5jbmVzLmZyL2F1dGgvcmVhbG1zL2h5c29wZSIsImF1ZCI6ImhuZy1hcGkiLCJzdWIiOiJlMWI4Mjc3Yi1jYjI5LTQxMTctYjNkOS01NmM4OTM0YzE3MGYiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJobmctYXBpIiwic2Vzc2lvbl9zdGF0ZSI6IjczN2IzODMxLTJiODctNDcwYy04YmM4LWFhYWJkODhiNjU0MSIsImFjciI6IjEiLCJhbGxvd2VkLW9yaWdpbnMiOltdLCJzY29wZSI6Im1pY3JvcHJvZmlsZS1qd3QgcmVnYXJkc19zY29wZSBlbWFpbCIsInNpZCI6IjczN2IzODMxLTJiODctNDcwYy04YmM4LWFhYWJkODhiNjU0MSIsInVwbiI6Im1hdGhpbGRlLnNpbWVvbkBjbmVzLmZyIiwicm9sZSI6IlJFR0lTVEVSRURfVVNFUiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLC

['/work/EXPERT_CENTER/mwec/workspace/HR/tuto_ostst/data2/URN_FEATURE_DATA_hysope2_7df60ccf-2b93-388d-be2e-7734fa7fea99_V1-URN_FEATURE_DATA_hysope2_7df60ccf-2b93-388d-be2e-7734fa7fea99_V1']

### Search and download RiverSP products

In [21]:
geometry = {"lonmin": 1.2, "latmin": 43.3, "lonmax": 1.5, "latmax": 43.8}

search_criteria = {
    "provider": catalog,
    "productType": "SWOT_L2_HR_RIVERSP_NODE",
    "start": "2023-02-04T04:03:00",
    "end": "2023-10-07T04:30:00",
    "geom": geometry}

river_products= dag.search_all(**search_criteria)

2024-09-06 20:45:48,140 - eodag.core - INFO - Searching product type 'SWOT_L2_HR_RIVERSP_NODE' on provider: hydroweb_next
2024-09-06 20:45:48,141 - eodag.core - INFO - Iterate search over multiple pages: page #1
2024-09-06 20:45:48,142 - eodag.search.qssearch - INFO - Sending search request: https://hydroweb.next.theia-land.fr/api/v1/rs-catalog/stac/search
2024-09-06 20:45:49,069 - eodag.core - INFO - Found 103 result(s) on provider 'hydroweb_next'


In [23]:
search_criteria["productType"] =  "SWOT_L2_HR_RIVERSP_REACH"

river_reach_products= dag.search_all(**search_criteria)

2024-09-06 20:45:54,470 - eodag.core - INFO - Searching product type 'SWOT_L2_HR_RIVERSP_REACH' on provider: hydroweb_next
2024-09-06 20:45:54,471 - eodag.core - INFO - Iterate search over multiple pages: page #1
2024-09-06 20:45:54,473 - eodag.search.qssearch - INFO - Sending search request: https://hydroweb.next.theia-land.fr/api/v1/rs-catalog/stac/search
2024-09-06 20:45:55,153 - eodag.core - INFO - Found 104 result(s) on provider 'hydroweb_next'


In [24]:
# Create a layer that maps the products found
mp = display_search_zone(geometry)

folium.GeoJson(
    data=river_products,
    tooltip=folium.GeoJsonTooltip(
        fields=[
            "title",
            "spatial:pass_id"
        ]
    ),
).add_to(mp)
mp

Filter out search results
After exploring the map, we only want to keep the products having a pass_number equals to 16. Let's crunch the search results according to this specific property

In [25]:
from eodag.crunch import FilterProperty

filtered_river_products = river_products.crunch(
    FilterProperty({"spatial:pass_id": 16, "operator": "eq"})
)
filtered_river_reach_products = river_reach_products.crunch(
    FilterProperty({"spatial:pass_id": 16, "operator": "eq"})
)
print(f"{len(river_products) - len(filtered_river_products)} river node products were filtered out by the property filter.")
print(f"{len(river_reach_products) - len(filtered_river_reach_products)} river reach products were filtered out by the property filter.")

2024-09-06 20:46:13,936 - eodag.crunch.property - INFO - Finished filtering products. 92 resulting products
2024-09-06 20:46:13,937 - eodag.crunch.property - INFO - Finished filtering products. 93 resulting products


11 river node products were filtered out by the property filter.
11 river reach products were filtered out by the property filter.


In [ ]:
dag.download_all(
    filtered_river_products, asset=r"SWOT_L2_HR_RiverSP_.*.zip", outputs_prefix=workdir
)

In [ ]:
dag.download_all(
    filtered_river_reach_products, asset=r"SWOT_L2_HR_RiverSP_.*.zip", outputs_prefix=workdir
)

#### Unzip the products
For some products, such as the riverSP ones, it is necesary to unzip to get to the shapefiles.

In [27]:
for root, dir, files in os.walk(workdir):
    for file in files:
        if file.endswith("zip"):
            print(os.path.join(root, file))
            shutil.unpack_archive(os.path.join(root, file), workdir)

./data/URN_FEATURE_DATA_swot_605cb315-d1bd-3168-bf56-58090bc2f261_V2-URN_FEATURE_DATA_swot_605cb315-d1bd-3168-bf56-58090bc2f261_V2/SWOT_L2_HR_RiverSP_Reach_496_016_EU_20230420T073109_20230420T073113_PGC0_01.zip
./data/URN_FEATURE_DATA_swot_e64b8d3a-1a7b-396f-b978-570b30d2eb8f_V1-URN_FEATURE_DATA_swot_e64b8d3a-1a7b-396f-b978-570b30d2eb8f_V1/SWOT_L2_HR_RiverSP_Reach_548_016_EU_20230610T232355_20230610T232359_PGC0_01.zip
./data/URN_FEATURE_DATA_swot_7793be0b-033b-3ce1-bea5-3c6bbfcb0f89_V2-URN_FEATURE_DATA_swot_7793be0b-033b-3ce1-bea5-3c6bbfcb0f89_V2/SWOT_L2_HR_RiverSP_Reach_514_016_EU_20230508T044229_20230508T044234_PGC0_01.zip
./data/URN_FEATURE_DATA_swot_2e1bd8db-dd73-3bb2-b1fd-14ffb247d47e_V2-URN_FEATURE_DATA_swot_2e1bd8db-dd73-3bb2-b1fd-14ffb247d47e_V2/SWOT_L2_HR_RiverSP_Reach_519_016_EU_20230513T035538_20230513T035542_PGC0_01.zip
./data/URN_FEATURE_DATA_swot_cc6ee12f-33b0-352f-a827-71621d30f1d5_V2-URN_FEATURE_DATA_swot_cc6ee12f-33b0-352f-a827-71621d30f1d5_V2/SWOT_L2_HR_RiverSP_Reach_

## Get SLC products from po.daac
### With podaac-data-subscriber

- You will need to open a free account with [NASA Earthdata Login](https://urs.earthdata.nasa.gov/)
- then run pip install podaac-data-subscriber

Configure the authentification by adding the foloowing lines in your ~/.netrc file :

```
machine urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```


Estimate the amount of file to download

In [ ]:
%%bash
podaac-data-downloader --dry-run --verbose -c SWOT_L1B_HR_SLC_2.0 -gr 'SWOT_L1B_HR_SLC_01?_320_078R_*' -e .nc -d ./data

Run and download

In [ ]:
%%bash
podaac-data-downloader --verbose -c SWOT_L1B_HR_SLC_2.0 -gr 'SWOT_L1B_HR_SLC_01?_320_078R_*' -e .nc -d /work/scratch/data/$USER

In [ ]:
%%bash
podaac-data-downloader --verbose -c SWOT_L2_HR_RiverSP_2.0 -gr 'SWOT_L1B_HR_RiverSP_5??_016_*' -e .nc -d /work/scratch/data/$USER --dry-run

### With earthaccess

In [ ]:
import earthaccess

print(earthaccess.__version__)

In [ ]:
auth = earthaccess.login()

In [ ]:
results = earthaccess.search_data(
    short_name="SWOT_L1B_HR_SLC_2.0",
    temporal=("2024-02-05 00:00:00", "2024-07-21 23:59:59"),
    granule_name="*_320_078R_*",
)

In [ ]:
files = earthaccess.download(results, workdir)